In [24]:
import numpy as np
import pandas as pd

import time
from pathos.multiprocessing import ProcessingPool as Pool
# from multiprocessing import Pool
from tqdm.notebook import tqdm
import hashlib
import json
import glob
import urllib
import urllib.request as urllib2
import requests
import http
import langid

from goose3 import Goose
from goose3.text import StopWordsChinese, StopWords, StopWordsArabic, StopWordsKorean

import warnings
warnings.filterwarnings('ignore')
print('The script still runs.')


The script still runs.


# event data & GKG data merging

In [ ]:
cnt = 0
for y in range(2023, 2025):
    files = glob.glob('gkg_f1_mining_water\\%s*'%y) # event and mine
    data = pd.DataFrame()
    for f in files:
        print('gkg:', f)
        temp = pd.read_csv(f)
        data = pd.concat([data, temp])
    
    files2 = []
    for x in range(y-1, y+2):
        files2 += glob.glob('event_mining_data\\%s*'%x)
    for f2 in files2:
        print('event:', f2)
        temp = pd.read_csv(f2)
        res = pd.merge(data, temp, how='inner', left_on='DocumentIdentifier', right_on='SOURCEURL').drop_duplicates()
        cnt += res.shape[0]
        res.to_csv(f2.replace('event_mining_data', 'gkg_event_mining_water_merged').replace('.CSV', '_'+str(y)+'.csv'), index=None)

files = glob.glob('gkg_event_mining_water_merged/*')
res = pd.DataFrame()
for f in files:
    temp = pd.read_csv(f)
    res = pd.concat([res,temp])
    print(f, temp.shape)
res = res.drop_duplicates()


In [ ]:
res['GKGRECORDID'].map(lambda x: x[:4]).value_counts()


In [ ]:
res.shape


# data screening and data exploding

In [30]:
res['isnull'] = res.isnull().mean(axis=1)
res = res.sort_values(by='isnull', ascending=False)
res = res.drop_duplicates(['GKGRECORDID', 'prop_id'], keep='last')
res.shape


(981549, 90)

In [31]:
res['V2Themes_lst'] = res['V2Themes'].map(lambda x:np.unique([i.split(',')[0] for i in  x.split(';')]))


In [33]:
water_theme_list = ['NATURAL_DISASTER_FLASH_FLOOD',
                    'NATURAL_DISASTER_FLASH_FLOODS',
                    'NATURAL_DISASTER_FLOOD',
                    'NATURAL_DISASTER_FLOOD_WARNING',
                    'NATURAL_DISASTER_FLOOD_WATER',
                    'NATURAL_DISASTER_FLOOD_WATERS',
                    'NATURAL_DISASTER_FLOODED',
                    'NATURAL_DISASTER_FLOODED_AREAS',
                    'NATURAL_DISASTER_FLOODING',
                    'NATURAL_DISASTER_FLOODS',
                    'NATURAL_DISASTER_FLOODWATER',
                    'NATURAL_DISASTER_FLOODWATERS',
                    'WB_154_FLOOD_PROTECTION',
                    'NATURAL_DISASTER_HIGH_WATER',
                    'NATURAL_DISASTER_HIGH_WATERS',
                    'NATURAL_DISASTER_WATER_LEVEL',
                    'WATER_SECURITY',
                    'CRISISLEX_C06_WATER_SANITATION',
                    'NATURAL_DISASTER_DROUGHT',
                    'NATURAL_DISASTER_DROUGHTS',
                    'NATURAL_DISASTER_INTENSE_RAINFALL',
                    'NATURAL_DISASTER_TORRENTIAL_RAINFALL',
                    'NATURAL_DISASTER_DESERTIFICATION',
                    'WATER_SECURITY',
                'WB_1000_WATER_MANAGEMENT_STRUCTURES',
                'WB_1002_IRRIGATION_WATER_QUALITY',
                'WB_1021_WATER_LAW',
                'WB_1063_WATER_ALLOCATION_AND_WATER_SUPPLY',
                'WB_1064_WATER_DEMAND_MANAGEMENT',
                'WB_1199_WATER_SUPPLY_AND_SANITATION',
                'WB_1215_WATER_QUALITY_STANDARDS',
                'WB_1220_SURFACE_WATER_MANAGEMENT',
                'WB_137_WATER',
                'WB_138_WATER_SUPPLY',
                'WB_139_SANITATION_AND_WASTEWATER',
                'WB_140_AGRICULTURAL_WATER_MANAGEMENT',
                'WB_141_WATER_RESOURCES_MANAGEMENT',
                'WB_142_ENERGY_AND_WATER',
                'WB_143_RURAL_WATER',
                'WB_144_URBAN_WATER',
                'WB_1462_WATER_SANITATION_AND_HYGIENE',
                'WB_149_WASTEWATER_TREATMENT_AND_DISPOSAL',
                'WB_150_WASTEWATER_REUSE',
                'WB_155_WATERSHED_MANAGEMENT',
                'WB_156_GROUNDWATER_MANAGEMENT',
                'WB_157_ENVIRONMENTAL_WATER_USE_AND_CATCHMENT_PROTECTION',
                'WB_158_WATER_RESOURCES_AND_CLIMATE_ADAPTATION',
                'WB_159_TRANSBOUNDARY_WATER',
                'WB_1729_URBAN_WATER_FINANCIAL_SUSTAINABILITY',
                'WB_1730_URBAN_WATER_COST_RECOVERY',
                'WB_1731_NON_REVENUE_WATER',
                'WB_1778_FRESHWATER_ECOSYSTEMS',
                'WB_1798_WATER_POLLUTION',
                'WB_1994_WATER_FOR_FUEL_EXTRACTION',
                'WB_1998_WATER_ECONOMICS',
                'WB_2005_COMMUNITY_WATER_SUPPLY_MANAGEMENT',
                'WB_2007_WATER_SAFETY_PLANS',
                'WB_2008_WATER_TREATMENT',
                'WB_2009_WATER_QUALITY_MONITORING',
                'WB_2968_URBAN_WATER_AND_SANITATION',
                'WB_2971_WATER_PRICING',
                'WB_2972_GROUNDWATER_CONJUNCTIVE_USE',
                'WB_2978_WATER_POLLUTION_LOAD',
                'WB_2981_DRINKING_WATER_QUALITY_STANDARDS',
                'WB_2982_INDUSTRIAL_WATER_AND_HYGIENE',
                'WB_2984_URBAN_WATER_AND_HYGIENE',
                'WB_2985_RURAL_WATER_AND_HYGIENE',
                'WB_2992_FRESHWATER_FISHERIES',
                'WB_3014_WASTEWATER_DISPOSAL_FACILITIES',
                'WB_423_INTEGRATED_URBAN_WATER_MANAGEMENT',
                'WB_427_WATER_ALLOCATION_AND_WATER_ECONOMICS',
                'WB_991_DRAINAGE_WATER_QUALITY',
                'WB_1710_DROUGHT_RISK_REDUCTION'
                   ]

water_theme_list = set(water_theme_list)
len(water_theme_list)


72

In [34]:
stat = pd.Series(lst).value_counts()


In [36]:
water_theme_dic = {
'natural_disaster_flood_warning':'Flooding and Excess Water',
'natural_disaster_flooded_areas':'Flooding and Excess Water',
'natural_disaster_torrential_rainfall':'Flooding and Excess Water',
'natural_disaster_flash_floods':'Flooding and Excess Water',
'natural_disaster_intense_rainfall':'Flooding and Excess Water',
'natural_disaster_flash_flood':'Flooding and Excess Water',
'natural_disaster_floodwaters':'Flooding and Excess Water',
'natural_disaster_floodwater':'Flooding and Excess Water',
'natural_disaster_floods':'Flooding and Excess Water',
'natural_disaster_flood_waters':'Flooding and Excess Water',
'natural_disaster_flooded':'Flooding and Excess Water',
'natural_disaster_flooding':'Flooding and Excess Water',
'natural_disaster_flood':'Flooding and Excess Water',
'natural_disaster_flood_water':'Flooding and Excess Water',
'natural_disaster_high_waters':'Flooding and Excess Water',
'natural_disaster_water_level':'Flooding and Excess Water',
'natural_disaster_high_water':'Flooding and Excess Water',
'natural_disaster_desertification':'Drought and Desertification',
'natural_disaster_droughts':'Drought and Desertification',
'natural_disaster_drought':'Drought and Desertification',
'wb_2981_drinking_water_quality_standards':'Water Quality and Pollution',
'wb_1798_water_pollution':'Water Quality and Pollution',
'wb_1215_water_quality_standards':'Water Quality and Pollution',
'wb_991_drainage_water_quality':'Water Quality and Pollution',
'wb_2009_water_quality_monitoring':'Water Quality and Pollution',
'wb_150_wastewater_reuse':'Water Supply, Sanitation and infrastructure',
'crisislex_c06_water_sanitation':'Water Supply, Sanitation and infrastructure',
'wb_149_wastewater_treatment_and_disposal':'Water Supply, Sanitation and infrastructure',
'wb_2005_community_water_supply_management':'Water Supply, Sanitation and infrastructure',
'wb_138_water_supply':'Water Supply, Sanitation and infrastructure',
'wb_1778_freshwater_ecosystems':'Water Supply, Sanitation and infrastructure',
'wb_139_sanitation_and_wastewater':'Water Supply, Sanitation and infrastructure',
'wb_143_rural_water':'Water Supply, Sanitation and infrastructure',
'wb_3014_wastewater_disposal_facilities':'Water Supply, Sanitation and infrastructure',
'wb_1462_water_sanitation_and_hygiene':'Water Supply, Sanitation and infrastructure',
'wb_1199_water_supply_and_sanitation':'Water Supply, Sanitation and infrastructure',
'wb_2008_water_treatment':'Water Supply, Sanitation and infrastructure',
'wb_1063_water_allocation_and_water_supply':'Water Supply, Sanitation and infrastructure',
'wb_154_flood_protection':'Water Supply, Sanitation and infrastructure',
'wb_144_urban_water':'Water Economics, Policy and Management',
'wb_1998_water_economics':'Water Economics, Policy and Management',
'wb_427_water_allocation_and_water_economics':'Water Economics, Policy and Management',
'wb_1064_water_demand_management':'Water Economics, Policy and Management',
'wb_1729_urban_water_financial_sustainability':'Water Economics, Policy and Management',
'wb_1731_non_revenue_water':'Water Economics, Policy and Management',
'wb_2971_water_pricing':'Water Economics, Policy and Management',
'water_security':'Water Economics, Policy and Management',
'wb_159_transboundary_water':'Water Economics, Policy and Management',
'wb_2007_water_safety_plans':'Water Economics, Policy and Management',
'wb_2992_freshwater_fisheries':'Water Economics, Policy and Management',
'wb_1021_water_law':'Water Economics, Policy and Management',
'wb_1000_water_management_structures':'Water Economics, Policy and Management',
'wb_141_water_resources_management':'Water Economics, Policy and Management',
'wb_140_agricultural_water_management':'Water Economics, Policy and Management',
'wb_156_groundwater_management':'Water Economics, Policy and Management',
'wb_155_watershed_management':'Water Economics, Policy and Management',
'wb_1220_surface_water_management':'Water Economics, Policy and Management',
'wb_1002_irrigation_water_quality':'Water Supply, Sanitation and infrastructure'}


In [37]:
ETMs = ['Copper', 'Palladium',
       'Nickel', 'Vanadium', 'Platinum',
       'Lead', 'Cobalt', 'Lithium',
       'Graphite', 'Manganese', 'Molybdenum',
       'Bauxite', 'Alumina', 'Aluminum',
       'Chromium', 'Platinum Group Metals']


In [38]:
res2 = res[['prop_name', 'prop_id', 'primary_commodity', 'dev_stage', 'actv_status',
           'country_name', 'snl_global_region',
           'latitude', 'longitude', 'ActionGeo_Long', 'ActionGeo_Lat',
#             'Actor1Name', 'Actor2Name', 'Actor1Type1Code', 'Actor2Type1Code', 'state_province',
            'AvgTone', 'Dist', 'DATEADDED',
            'V2Tone', 'V2Themes_lst', 'GKGRECORDID', 'GLOBALEVENTID']]


In [39]:
res2 = res2.explode('V2Themes_lst')


In [40]:
res2 = res2[res2['V2Themes_lst'].isin(water_theme_list)]


In [41]:
res2 = res2[res2['V2Themes_lst']!='wb_137_water']


In [42]:
res2.to_csv('mining_water_GKG_event.csv', index=None)


In [43]:
res2.shape


(4740384, 18)